In [57]:
import math
import os
import glob
import json
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread

## check one json

In [58]:
one_json_path = '/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_json_val/BLD00007_PS3_K3A_NIA0276.json'

# load json (one json)

In [59]:
with open(one_json_path) as f:
    json_file = json.load(f)
    print(json.dumps(json_file, indent=2))

{
  "features": [
    {
      "geometry": {
        "coordinates": [
          [
            31.4289436358,
            30.0362447983,
            0.0
          ],
          [
            31.4347830009,
            30.0363144197,
            0.0
          ],
          [
            31.4347031225,
            30.0413951468,
            0.0
          ],
          [
            31.4288634597,
            30.0413255113,
            0.0
          ]
        ],
        "type": "Polygon"
      },
      "properties": {
        "object_imcoords": "EMPTY",
        "building_imcoords": "597.7326163372675,1.489861954978234,598.6265335102545,18.77226063272575,620.6764904439323,20.262122587703985,620.0805456619411,14.89861954978234,636.7669995576972,17.878343459738808,636.4690271667016,0",
        "road_imcoords": "EMPTY",
        "image_id": "BLD00007_PS3_K3A_NIA0276.png",
        "ingest_time": "2020-09-08T07:18:00.810381Z",
        "type_id": "2",
        "type_name": "\uc544\ud30c\ud2b8"
      },

In [60]:
json_file.keys()

dict_keys(['features', 'type'])

# json check x,y (one json)

In [61]:
for feature in json_file['features']:
    print(list(map(float, feature['properties']['building_imcoords'].split(','))))
    print(feature['properties']['type_id'])
    print(feature['properties']['type_name'])
    print(feature['properties']['image_id'])
    print('------------------')

[597.7326163372675, 1.489861954978234, 598.6265335102545, 18.77226063272575, 620.6764904439323, 20.262122587703985, 620.0805456619411, 14.89861954978234, 636.7669995576972, 17.878343459738808, 636.4690271667016, 0.0]
2
아파트
BLD00007_PS3_K3A_NIA0276.png
------------------
[599.1921902597313, 18.927392965209417, 600.7694730068322, 52.92659884715966, 611.2846913208374, 64.14283171543191, 634.2429179730822, 69.75094814956803, 634.0676643345154, 60.98826622123033, 635.4696934430494, 17.875871133808893, 620.5731341648753, 16.123334748141353, 620.5731341648753, 20.15416843517669]
2
아파트
BLD00007_PS3_K3A_NIA0276.png
------------------
[643.3561071785533, 0.0, 643.3561071785533, 21.731451182277475, 674.200747566302, 21.03043662801046, 674.3760012048688, 9.98945739830497, 674.200747566302, 0.0]
2
아파트
BLD00007_PS3_K3A_NIA0276.png
------------------
[642.830346262853, 22.607719375111245, 644.4076290099539, 71.50348453523557, 660.5309637580953, 73.78178183660337, 671.2214357106672, 71.15297725810206,

# json 폴더 경로

In [62]:
json_folder = '/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_json_val/'

# make polygon list, label_list

In [63]:
def parse_json(json_file):
    polygon_list = []
    label_list = []
    for feature in json_file['features']:
        if feature['properties']['building_imcoords']:
            polygon = list(map(float, feature['properties']['building_imcoords'].split(',')))
            polygon = list(map(math.floor, polygon))
            polygon = np.array(polygon, np.int32).reshape(-1, 1, 2)
            polygon_list.append(polygon)
        
            label = int(feature['properties']['type_id'])
            label_list.append(label)
        
            if label not in label_dict:
                label_dict[label] = feature['properties']['type_name']
    
    return polygon_list, label_list

# read json folder

In [64]:
file_list = os.listdir(label_folder)
json_files = [file for file in file_list if file.endswith(".json")]

print ("file_list_py: {}".format(json_files))

file_list_py: ['BLD02499_PS3_K3A_NIA0375.json', 'BLD04051_PS3_K3A_NIA0377.json', 'BLD00710_PS3_K3A_NIA0277.json', 'BLD02274_PS3_K3A_NIA0374.json', 'OBJ03670_PS3_K3A_NIA0153.json', 'BLD00101_PS3_K3A_NIA0276.json', 'BLD00175_PS3_K3A_NIA0276.json', 'BLD00854_PS3_K3A_NIA0277.json', 'BLD10664_PS3_K3A_NIA0389.json', 'OBJ02974_PS3_K3_NIA0127.json', 'OBJ01050_PS3_K3_NIA0089.json', 'BLD00202_PS3_K3A_NIA0276.json', 'BLD00134_PS3_K3A_NIA0276.json', 'BLD00994_PS3_K3A_NIA0277.json', 'BLD01557_PS3_K3A_NIA0373.json', 'BLD02180_PS3_K3A_NIA0374.json', 'BLD00080_PS3_K3A_NIA0276.json', 'BLD01603_PS3_K3A_NIA0373.json', 'BLD00031_PS3_K3A_NIA0276.json', 'BLD03311_PS3_K3A_NIA0376.json', 'BLD00964_PS3_K3A_NIA0277.json', 'BLD01760_PS3_K3A_NIA0373.json', 'BLD12346_PS3_K3A_NIA0391.json', 'BLD06604_PS3_K3A_NIA0381.json', 'BLD00051_PS3_K3A_NIA0276.json', 'BLD01501_PS3_K3A_NIA0373.json', 'BLD01037_PS3_K3A_NIA0277.json', 'BLD00174_PS3_K3A_NIA0276.json', 'BLD11632_PS3_K3A_NIA0390.json', 'OBJ01226_PS3_K3_NIA0089.json'

In [65]:
json_files.sort()

In [66]:
json_files

['BLD00007_PS3_K3A_NIA0276.json',
 'BLD00031_PS3_K3A_NIA0276.json',
 'BLD00033_PS3_K3A_NIA0276.json',
 'BLD00051_PS3_K3A_NIA0276.json',
 'BLD00080_PS3_K3A_NIA0276.json',
 'BLD00096_PS3_K3A_NIA0276.json',
 'BLD00101_PS3_K3A_NIA0276.json',
 'BLD00126_PS3_K3A_NIA0276.json',
 'BLD00134_PS3_K3A_NIA0276.json',
 'BLD00148_PS3_K3A_NIA0276.json',
 'BLD00155_PS3_K3A_NIA0276.json',
 'BLD00174_PS3_K3A_NIA0276.json',
 'BLD00175_PS3_K3A_NIA0276.json',
 'BLD00201_PS3_K3A_NIA0276.json',
 'BLD00202_PS3_K3A_NIA0276.json',
 'BLD00220_PS3_K3A_NIA0276.json',
 'BLD00576_PS3_K3A_NIA0277.json',
 'BLD00592_PS3_K3A_NIA0277.json',
 'BLD00595_PS3_K3A_NIA0277.json',
 'BLD00618_PS3_K3A_NIA0277.json',
 'BLD00652_PS3_K3A_NIA0277.json',
 'BLD00682_PS3_K3A_NIA0277.json',
 'BLD00710_PS3_K3A_NIA0277.json',
 'BLD00712_PS3_K3A_NIA0277.json',
 'BLD00811_PS3_K3A_NIA0277.json',
 'BLD00854_PS3_K3A_NIA0277.json',
 'BLD00945_PS3_K3A_NIA0277.json',
 'BLD00961_PS3_K3A_NIA0277.json',
 'BLD00964_PS3_K3A_NIA0277.json',
 'BLD00981_PS3

# json to png

In [67]:
def draw_label(json_path, polygon_list):
        filename = os.path.splitext(os.path.basename(json_path))[0]
        savepath = '/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_png_sample/label/'
        image = np.zeros((1024, 1024, 3), np.uint8)
        img = cv2.fillPoly(image, polygon_list, (128, 128, 128))
        img = Image.fromarray(img)
        img.save(f'{savepath}{filename}.png')
        print(f'{savepath}{filename}.png saved!')

In [69]:
for i in range(len(json_files)):
    json_name = json_files[i]
    print(json_path)
    json_path = json_folder + json_name
    with open(json_path) as f:
        
        json_file = json.load(f)
        polygon_list, label_list = parse_json(json_file)
        draw_label(json_path, polygon_list)


BLD00007_PS3_K3A_NIA0276.json
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_png_sample/label/BLD00007_PS3_K3A_NIA0276.png saved!
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_json_val/BLD00007_PS3_K3A_NIA0276.json
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_png_sample/label/BLD00031_PS3_K3A_NIA0276.png saved!
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_json_val/BLD00031_PS3_K3A_NIA0276.json
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_png_sample/label/BLD00033_PS3_K3A_NIA0276.png saved!
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_json_val/BLD00033_PS3_K3A_NIA0276.json
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_png_sample/label/BLD00051_PS3_K3A_NIA0276.png saved!
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_json_val/BLD00051_PS3_K3A_NIA0276.json
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_png_sample/label/BLD00080_PS3_K3A_NIA0276.png saved!
/home/aiffel-dj17/

/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_png_sample/label/BLD01501_PS3_K3A_NIA0373.png saved!
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_json_val/BLD01501_PS3_K3A_NIA0373.json
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_png_sample/label/BLD01545_PS3_K3A_NIA0373.png saved!
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_json_val/BLD01545_PS3_K3A_NIA0373.json
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_png_sample/label/BLD01548_PS3_K3A_NIA0373.png saved!
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_json_val/BLD01548_PS3_K3A_NIA0373.json
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_png_sample/label/BLD01556_PS3_K3A_NIA0373.png saved!
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_json_val/BLD01556_PS3_K3A_NIA0373.json
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sample/building_png_sample/label/BLD01557_PS3_K3A_NIA0373.png saved!
/home/aiffel-dj17/aiffel/SIM2DATA/json2png_sampl